In [41]:
import os

In [42]:
%pwd

'c:\\End to End Machine Learning project with mlflow'

In [44]:
os.chdir("../")

In [43]:
%pwd

'c:\\End to End Machine Learning project with mlflow'

In [45]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [46]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [53]:
#CONFIG_FILE_PATH = "C:\End to End Machine Learning project with mlflow\End-to-End-Machine-Learning-project-with-mlflow\config\config.yaml"
#PARAMS_FILE_PATH = "C:\End to End Machine Learning project with mlflow\End-to-End-Machine-Learning-project-with-mlflow\params.yaml"
#SCHEMA_FILE_PATH = "C:\End to End Machine Learning project with mlflow\End-to-End-Machine-Learning-project-with-mlflow\schema.yaml"
from pathlib import Path

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        # Convert string paths to pathlib.Path objects
        config_filepath = Path(config_filepath)
        params_filepath = Path(params_filepath)
        schema_filepath = Path(schema_filepath)
        
        # Use the converted paths in the read_yaml function
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [54]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [55]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [56]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-29 18:32:40,646: INFO: common: yaml file: C:\End to End Machine Learning project with mlflow\End-to-End-Machine-Learning-project-with-mlflow\config\config.yaml loaded successfully]
[2024-08-29 18:32:40,657: INFO: common: yaml file: C:\End to End Machine Learning project with mlflow\End-to-End-Machine-Learning-project-with-mlflow\params.yaml loaded successfully]
[2024-08-29 18:32:40,665: INFO: common: yaml file: C:\End to End Machine Learning project with mlflow\End-to-End-Machine-Learning-project-with-mlflow\schema.yaml loaded successfully]
[2024-08-29 18:32:40,667: INFO: common: created directory at: artifacts]
[2024-08-29 18:32:40,669: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-29 18:32:42,646: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/